In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import json

In [ ]:
files = {
    './193722ad5ac72833_c_28d755665b1acbdc.htm': 'colonnes',
    './26a4311a5ac7ebb2_c_28d75e255b1a3ac6.htm': 'tables',
    './26b0b12d5ad07d00_c_28d754e35b1a6fdc.htm': 'bases',
    './755c7dc65ae16106_c_28d7554b5b1a3192.htm': 'cles',
    './a4b7af9d5af204b7_c_28d754e45b1a71ce.htm': 'id_cles',
    './a4b7b3555af20850_c_28d755225b1ae133.htm': 'cle_migrantes',
    './a4b7b4da5af20922_c_28d75e165b1a047d.htm': 'cle_colonne'
}

In [ ]:
for filename, name in files.items():
    print(f"Extracting {name} ... ", end="")
    with open(filename, encoding="latin-1") as f:
        html =  f.readlines()
    
    soup = BeautifulSoup(str(html), 'lxml')
    table = soup.find("table")
    
    headings = [th.get_text().rstrip().lstrip().lower().replace(' ', '_') for th in table.find("tr").find_all("th")]
    headings = [s.replace('é', 'e') for s in headings]
    
    datasets = []
    for row in table.find_all("tr")[1:]:
        dataset = dict(zip(headings, (td.get_text().rstrip().lstrip() for td in row.find_all("td"))))
        datasets.append(dataset)
    
    data = pd.DataFrame.from_records(datasets)
    
    with open(f'./{name}.json', 'w') as f:
        data.to_json(f, orient='records')
        
    dicts = data.to_dict(orient='records')
    final_json_string = ''
    for i, document in enumerate(dicts):
        metadata = {"index": {"_id": i}}
        final_json_string += json.dumps(metadata) + '\n' + json.dumps(document) + '\n'
    
    with open(f'./{name}_elastic.json', 'w') as f:
        f.write(final_json_string)

    print(f" done. (written to {name}.json & {name}_elastic.json)")